In [1]:
import numpy as np
import os
import datetime
import os
from PIL import Image, ImageFilter, ImageEnhance
import imageio
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
import tensorflow as tf
tf.random.set_seed(30)

In [3]:
# access the dataset from my google drive
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# added this piece later as it was throwing error - the backend to open the image files with iomode ri is not available.
#!pip install imageio[pyav]


In [5]:
# Access the file
train_doc = np.random.permutation(open('/content/drive/MyDrive/Datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/drive/MyDrive/Datasets/Project_data/val.csv').readlines())

# multiple options are tried out
batch_size = 8  # experiment with the batch size

In [6]:
# optional code

In [7]:
# Defining the generator
def generator(source_path, folder_list, batch_size):
    #print('Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28] # create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(source_path)//batch_size  # calculate the number of batches
        ######
        ### Remove above piece
        ######

        #num_batches = len(folder_list)//batch_size  # calculate the number of batches

        for batch in range(num_batches):  # we iterate over the number of batches
            # first try with x=20
            batch_data = np.zeros((batch_size, 18, 100, 100, 3))  # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size, 5))  # batch_labels is the one hot representation of the output

            for folder in range(batch_size):  # iterate over the batch_size
                #print('Source path = ',source_path)
                #print('\n')
                #print("Folder :",folder)
                #print("\n")
                #print("files :",t[folder + (batch*batch_size)].split(';')[0])
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])  # read all the images in the folder
                for idx, item in enumerate(img_idx):  # Iterate over the frames/images of a folder to read them in
                    image_path = os.path.join(source_path, t[folder + (batch * batch_size)].strip().split(';')[0], imgs[item])
                    image = imageio.imread(image_path).astype(np.float32)

                    # Resize the image
                    image = cv2.resize(image, (100, 100))

                    # Normalize and feed in the image
                    batch_data[folder, idx, :, :, 0] = image[:, :, 0] / 255
                    batch_data[folder, idx, :, :, 1] = image[:, :, 1] / 255
                    batch_data[folder, idx, :, :, 2] = image[:, :, 2] / 255

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

        # write the code for the remaining data points which are left after full batches
        if (len(source_path) % batch_size) // 2 == 0:
            batch_size = 2
        else:
            batch_size = 1
        num_batches = len(source_path) % batch_size  # calculate the number of batches
        for batch in range(num_batches):  # we iterate over the number of batches
            batch_data = np.zeros((batch_size, 18, 100, 100, 3))  # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size, 5))  # batch_labels is the one hot representation of the output
            for folder in range(batch_size):  # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])  # read all the images in the folder
                for idx, item in enumerate(img_idx):  # Iterate over the frames/images of a folder to read them in
                    image_path = os.path.join(source_path, t[folder + (batch * batch_size)].strip().split(';')[0], imgs[item])
                    image = imageio.imread(image_path).astype(np.float32)

                    # Resize the image
                    image = cv2.resize(image, (100, 100))

                    # Normalize and feed in the image
                    batch_data[folder, idx, :, :, 0] = image[:, :, 0] / 255
                    batch_data[folder, idx, :, :, 1] = image[:, :, 1] / 255
                    batch_data[folder, idx, :, :, 2] = image[:, :, 2] / 255

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [8]:
curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 25  # choose the number of epochs
print('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 25


In [9]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

Input_shape = (18, 100, 100, 3)

model = Sequential()

model.add(Conv3D(8, (3, 3, 3), padding='same', input_shape=Input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(16, (3, 3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dropout(0.5))



model.add(Conv3D(32, (3, 3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dropout(0.5))

model.add(Conv3D(64, (3, 3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(0.5))

#model.add(Conv3D(64, (3, 3, 3)))
#model.add(Activation('relu'))
#model.add(BatchNormalization())
#model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dropout(0.5))

model.add(Flatten())

#model.add(Dense(64))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))


model.add(Dense(5))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Activation('softmax'))

In [10]:
#model.build(input_shape=(batch_size,) + Input_shape)

#optimiser = optimizers.Adam(learning_rate=0.001)
optimiser = optimizers.SGD(learning_rate=0.001, momentum=0.7, nesterov=True)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Print the model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 18, 100, 100, 8)   656       
                                                                 
 batch_normalization (Batch  (None, 18, 100, 100, 8)   32        
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 18, 100, 100, 8)   0         
                                                                 
 max_pooling3d (MaxPooling3  (None, 9, 50, 50, 8)      0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (None, 9, 50, 50, 16)     3472      
                                                                 
 batch_normalization_1 (Bat  (None, 9, 50, 50, 16)     6

In [11]:
# Create the generators
train_generator = generator('/content/drive/MyDrive/Datasets/Project_data/train', train_doc, batch_size)
val_generator = generator('/content/drive/MyDrive/Datasets/Project_data/val', val_doc, batch_size)

In [12]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ', '').replace(':', '_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

# filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq="epoch")

LR = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1, mode="auto", min_delta=1e-04, cooldown=1, min_lr=0)
callbacks_list = [checkpoint, LR]

In [13]:
# Calculate steps per epoch and validation steps
if (num_train_sequences % batch_size) == 0:
    steps_per_epoch = int(num_train_sequences / batch_size)
else:
    steps_per_epoch = (num_train_sequences // batch_size) + 1

if (num_val_sequences % batch_size) == 0:
    validation_steps = int(num_val_sequences / batch_size)
else:
    validation_steps = (num_val_sequences // batch_size) + 1

In [14]:
# Fit the model
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/25
83/83 [==============================] - ETA: 0s - loss: 1.6415 - categorical_accuracy: 0.1908 
Epoch 1: saving model to model_init_2024-04-0318_37_42.639787/model-00001-1.64146-0.19079.h5
83/83 [==============================] - 1680s 20s/step - loss: 1.6415 - categorical_accuracy: 0.1908 - val_loss: 1.6094 - val_categorical_accuracy: 0.1613 - lr: 0.0010
Epoch 2/25
66/83 [======================>.......] - ETA: 3:08 - loss: 1.6094 - categorical_accuracy: 0.2652

InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) INVALID_ARGUMENT:  ValueError: Could not find a backend to open `/content/drive/MyDrive/Datasets/Project_data/train/WIN_20180907_16_05_58_Pro_Stop Gesture_new/WIN_20180907_16_05_58_Pro_00032.png`` with iomode `ri`.
Based on the extension, the following plugins might add capable backends:
  pyav:  pip install imageio[pyav]
Traceback (most recent call last):

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in gene